<img src="../DLSU-ALTDSI-logo.png" width="100%" style="margin-bottom:-40px; margin-top:-30px;"/>

**This notebook contains the context-aware video retrieval pipeline used in the study:**

## *Comparing Modality Representation Schemes in Video Retrieval for More Context-Aware Auto-Annotation of Trending Short-Form Videos*

**By the following researchers from the Andrew L. Tan Data Science Institute:**
1. Ong, Matthew Kristoffer Y. (matthew_kristoffer_ong@dlsu.edu.ph)
2. Presas, Shanette Giane G. (shanette_giane_presas@dlsu.edu.ph)
3. Sarreal, Sophia Althea R. (sophia_sarreal@dlsu.edu.ph)
4. To, Jersey Jaclyn K. (jers_to@dlsu.edu.ph)

---

## Dependencies

In [39]:
# Run this to activate venv for the terminal instance: .venv\Scripts\activate

import os
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import ffmpeg
import torch
import librosa
from panns_inference import AudioTagging

# Make sure cuda (gpu) is active!
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


---
## **AUDIO MODALITY**
**Goal**: Produce embeddings representing the audio modality of a given set of videos.

**Preprocessing step:** extract 32kHz waveform files from the input videos.

In [40]:
def extract_audio_to_wavs(video_path: str, out32: str, overwrite: bool=True):
    extract_32k=(
        ffmpeg.input(video_path).output(out32, format='wav', acodec='pcm_s16le', ac=1, ar=32000)
    )
    if overwrite:
        extract_32k = extract_32k.overwrite_output()
    
    extract_32k.run(quiet=True)
    print("Wrote 32kHz", out32)

In [41]:
def process_video(video_path: str, out_dir: str ="proc_out"):
    out_dir = Path(out_dir)
    audio_dir = out_dir.parent / (out_dir.name + "_32kHz")
    audio_dir.mkdir(parents=True, exist_ok=True) # 32kHz goes to audio_dir

    video = Path(video_path)
    out32 = audio_dir / (video.stem + "_32k.wav") # 32kHz output

    # Extract audio
    extract_audio_to_wavs(str(video), str(out32))

In [42]:
media_dir = Path("media")
videos = list(media_dir.glob("*.mp4"))
print(f"{len(videos)} videos found!")

for video in videos:
    print(f"\nProcessing: {video.name}")
    process_video(video)

6 videos found!

Processing: trend3vid6.mp4
Wrote 32kHz proc_out_32kHz\trend3vid6_32k.wav

Processing: trend3vid7.mp4
Wrote 32kHz proc_out_32kHz\trend3vid7_32k.wav

Processing: trend3vid8.mp4
Wrote 32kHz proc_out_32kHz\trend3vid8_32k.wav

Processing: trend5vid2.mp4
Wrote 32kHz proc_out_32kHz\trend5vid2_32k.wav

Processing: trend5vid3.mp4
Wrote 32kHz proc_out_32kHz\trend5vid3_32k.wav

Processing: trend5vid4.mp4
Wrote 32kHz proc_out_32kHz\trend5vid4_32k.wav


**Feature extraction step:** produce embeddings in the form of a 2048-dimensional feature vector representing the audio of the videos.

In [43]:
proc_out_32kHz_dir = Path("proc_out_32kHz")
emb_out_dir = Path("embeddings_out/audio2048") # 2048-d vectors go here
emb_out_dir.mkdir(parents=True, exist_ok=True)

at_model = AudioTagging(checkpoint_path=None, device=device) #this is the pretrained CNN14

wav_files = sorted(proc_out_32kHz_dir.glob("*_32k.wav"))
print(f"{len(wav_files)} WAV files found!")

for wav_path in wav_files:
    print(f"\nProcessing: {wav_path.name}")
    wav, sr = librosa.load(str(wav_path), sr=32000, mono=True) # just to make sure wav is 32kHz
    audio_batch = np.expand_dims(wav, axis=0) # matches the expected shape of PANN

    _, embedding = at_model.inference(audio_batch) # gets the embedding as numpy array

    embedding_vec = embedding[0] # first element of embedding array

    out_path = emb_out_dir / (wav_path.stem + "_embedding2048.npy")
    np.save(str(out_path), embedding_vec)
    print("Embedding saved: ", out_path)

    print(embedding_vec) # if you want to see the vector
    print(embedding_vec.shape)

Checkpoint path: C:\Users\mkyod/panns_data/Cnn14_mAP=0.431.pth
GPU number: 1
6 WAV files found!

Processing: trend3vid6_32k.wav
Embedding saved:  embeddings_out\audio2048\trend3vid6_32k_embedding2048.npy
[0.         0.         0.         ... 0.19923183 0.46746948 0.        ]
(2048,)

Processing: trend3vid7_32k.wav
Embedding saved:  embeddings_out\audio2048\trend3vid7_32k_embedding2048.npy
[0.        0.        0.        ... 0.        0.8063234 0.       ]
(2048,)

Processing: trend3vid8_32k.wav
Embedding saved:  embeddings_out\audio2048\trend3vid8_32k_embedding2048.npy
[0.         0.         0.         ... 0.5110518  0.74246013 0.        ]
(2048,)

Processing: trend5vid2_32k.wav
Embedding saved:  embeddings_out\audio2048\trend5vid2_32k_embedding2048.npy
[0.         0.         0.         ... 0.19942343 0.04068661 0.        ]
(2048,)

Processing: trend5vid3_32k.wav
Embedding saved:  embeddings_out\audio2048\trend5vid3_32k_embedding2048.npy
[0.         0.         0.         ... 0.22457194 0